# 개요
* GPT활용교육을 들은 내용을 별도로 정리
  * 1일차 : 네이버API 활용 : GPT를 활용한 변수 분석 등
  * 2일차 : beautifulsoup활용한 뉴스 및 증권사리포트 크롤링
  * 3일차 : 뉴스 및 증권사리포트 데이터 전처리 
  * 4일차 : 감성점수 변환
  * 5일차 : 백테스팅 실습

# 2일차 실습내용 정리

## 실습 : BeautifulSoup 객체생성 및 태크/클래스로 크롤링

In [ ]:
from bs4 import BeautifulSoup

html_sample = """
<html>
  <head>
    <title>테스트 페이지</title>
  </head>
  <body>
    <h1 id="main-title">hello, world</h1>
    <p class="description">웹페이지 크롤링중</p>
    <a href="https://example.com" class="link">example</a>
  </body>
</html>
"""

# BeautifulSoup 객체생성
soup = BeautifulSoup(html_sample)

In [ ]:
## 제목 태그 (title)
title_tag = soup.title

print(title_tag)
print(title_tag.text)

<title>테스트 페이지</title>
테스트 페이지


In [ ]:
## 특정 태그 (h1)
h1 = soup.find('h1')

print(h1)
print(h1.text)

<h1 id="main-title">hello, world</h1>
hello, world


In [ ]:
## 클래스 (description)
description = soup.find('p', class_='description')

print(description)
print(description.text)

<p class="description">웹페이지 크롤링중</p>
웹페이지 크롤링중


In [ ]:
## 태그 속성 접근 (href)
link = soup.find('a')

print(link)
print(link['href'])

<a class="link" href="https://example.com">example</a>
https://example.com


In [ ]:
## 모든 태그 찾기
all_links = soup.find_all('a')

print(all_links, '\n')

for link in all_links:
    print(link['href'])

[<a class="link" href="https://example.com">example</a>] 

https://example.com


## requests와 함께 사용하는 샘플 예제

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://example.com'
response = requests.get(url)

soup = BeautifulSoup(response.text)

## 실습 : GPT를 활용한 뉴스크롤링

* GPT에 아래와 같은 프롬프트를 입력 (좋은 값을 뽑기위해 먼저 GPT가 생각하게 함)

    ```
    url과 html을 참고해서, url에 어떤 값을 줄 수 있고, html에서 어떤 걸 크롤링할 수 있는지 고민

    url : https://search.naver.com/search.naver?ssc=tab.news.all&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2023.01.01&de=2023.02.27&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230101to20230131&is_sug_officeid=0&office_category=&service_area=

    html : <copyElement로 긁어낸 실제 html을 복사. 너무 길어서 여기서는 생략. (copyElement는 하위항목의 태그를 포함하여 복사함)>
    ```
* GPT가 URL분석(검색 파라미터의 의미)과 html에서 추출가능한 데이터를 알려줌 (하단은 결과샘플)
  * URL분석
    ```
    https://search.naver.com/search.naver?ssc=tab.news.all&query=삼성전자&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2023.01.01&de=2023.02.27&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from20230101to20230131&is_sug_officeid=0&office_category=&service_area=

    ```
  * 결과샘플

    | 파라미터                                                                                                                                                     | 의미                                      |
    | -------------- | --------------------------------------- |
    | `query`                                                                                                                                                  | 검색 키워드 (`삼성전자`)                         |
    | `sort`                                                                                                                                                   | 정렬 기준 (0: 관련도, 1: 최신순, 2: 오래된순)         |
    | `pd`                                                                                                                                                     | 날짜 범위 (0: 전체, 1: 1일, 2: 1주일, 3: 직접입력)   |
    | `ds` / `de`                                                                                                                                              | 시작일 (`2023.01.01`) / 종료일 (`2023.02.27`) |
    | `nso`                                                                                                                                                    | 날짜 범위 정제 필터, `p:from~to` 형식             |
    | `photo`, `field`, `mynews`, <br>`office_type`, `related`, <br>`office_section_code`, <br>`news_office_checked`,<br> `office_category`, <br>`service_area`, <br>`is_sug_officeid` | 검색 필터 관련 옵션 (필요 시 확장 가능)                |
  * HTML에서 크롤링 가능한 정보

    | 정보 종류    | 설명                       | 예시                                             |
    | -------------- | ------------------------ | ---------------------------------------------- |
    | 언론사      | 기사 발행 언론사 이름             | `조선비즈`, `이데일리` 등                               |
    | 날짜       | 기사 발행일                   | `2023.01.01.`                                  |
    | 제목       | 기사 제목 텍스트                | `"삼성·LG전자 4분기 실적발표"`                           |
    | 요약       | 기사 요약 텍스트                | `"삼성전자와 LG전자의 4분기 잠정실적..."`                    |
    | 기사 원문 링크 | `href` 속성에 포함된 뉴스 원문 URL | `https://biz.chosun.com/...`                   |
    | 썸네일 이미지  | 이미지 링크                   | `https://imgnews.pstatic.net/image/origin/...` |

* GPT에 아래와 같은 프롬프트를 입력 (샘플 코드 생성)

    ```
    아래 조건에 맞는 python requests, bs4를 사용한 코드 작성
    - url에서 query랑 ds, de를 값으로 주기
    - html에서는 기사 제목, 요약, 언론사이름, 날짜를 크롤링
    ```

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import datetime

def get_news(query: str, ds: str, de: str, display_cnt: int = 1):
    """
    네이버 뉴스 검색에서 기사 제목, 요약, 언론사 이름, 날짜 크롤링

    Parameters:
        query (str): 검색어
        ds (str): 시작일자 ('2025.05.01' 형식)
        de (str): 종료일자 ('2025.05.22' 형식)
        display_cnt (int): 가져올 페이지 수 (1페이지 = 10건)
    """
    base_url = 'https://search.naver.com/search.naver'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
        'Referer': 'https://www.naver.com/'
    }

    all_data = []
    
    for page in range(display_cnt):
        params = {
            'where': 'news',
            'query': query,
            'sm': 'tab_opt',
            'sort': 0,
            'photo': 0,
            'field': 0,
            'pd': 3,  # 기간 직접 설정
            'ds': ds,
            'de': de,
            'start': page * 10 + 1
        }

        response = requests.get(base_url, headers=headers, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')

        news_list = soup.select('div.sds-comps-full-layout.fender-news-item-list-tab > div > div[data-sds-comp="Profile"]')

        for news in news_list:
            title_tag = news.select_one('a.news_tit')
            summary_tag = news.select_one('div.news_dsc')
            press_tag = news.select_one('a.info_group > span.press')
            date_tag = news.select_one('span.info')

            title = title_tag.get_text(strip=True) if title_tag else ''
            link = title_tag['href'] if title_tag and title_tag.has_attr('href') else ''
            summary = summary_tag.get_text(strip=True) if summary_tag else ''
            press = press_tag.get_text(strip=True) if press_tag else ''
            date = date_tag.get_text(strip=True) if date_tag else ''

            all_data.append({
                '제목': title,
                '요약': summary,
                '언론사': press,
                '날짜': date,
                '링크': link
            })
    
    return all_data

# 예시 사용
if __name__ == '__main__':
    query = '삼성전자'
    ds = '2025.05.01'
    de = '2025.05.01'
    result = get_news(query, ds, de, display_cnt=2)

    for idx, item in enumerate(result, 1):
        print(f"\n[{idx}]")
        for key, value in item.items():
            print(f"{key}: {value}")


## 유의사항

* 네이버에는 로봇 차단 방지 기능이 있음
  * User-Agent설정, 요청간간딜레이를 추가하여 대응

* 데이터 사용 윤리
  * 공개된 데이터만 활용하고, 과도한 요청 피하기

## 처음 제공된 기초 코드 샘플

In [ ]:
from pykrx import stock

# 코스피 상장 종목 코드 리스트 가져오기
tickers = stock.get_market_ticker_list(market="KOSPI")

kospi_name = []

# 종목 코드와 종목명 출력
for ticker in tickers:
    name = stock.get_market_ticker_name(ticker)
    kospi_name.append(name)

print(len(kospi_name))

963


In [ ]:
from tqdm import tqdm
import pandas as pd

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import time

def get_naver_news_titles(company_name, start_date, end_date):
    # 네이버 뉴스 검색 URL 생성
    base_url = "https://search.naver.com/search.naver"
    query = urllib.parse.quote(company_name)
    start_date_clean = start_date.replace('.', '')
    end_date_clean = end_date.replace('.', '')

    params = {
        'where': 'news',
        'query': query,
        'sm': 'tab_opt',
        'sort': '0',
        'photo': '0',
        'field': '0',
        'pd': '3',
        'ds': start_date,
        'de': end_date,
        'nso': f'so:r,p:from{start_date_clean}to{end_date_clean}',
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36',
        'Referer': 'https://www.naver.com/'
    }

    # 세션 객체 생성
    session = requests.Session()

    try:
        response = session.get(base_url, params=params, headers=headers)

        # 응답 상태 코드 확인
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            return []

        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')
        news_data = []

        # 뉴스 제목, 날짜, 요약 찾기
        for news in soup.select("div.news_area"):
            title_tag = news.select_one("a.news_tit")
            date_tag = news.select_one("span.info")
            summary_tag = news.select_one("div.dsc_wrap")

            if title_tag and date_tag and summary_tag:
                title = title_tag.get('title')
                date = date_tag.text
                summary = summary_tag.text.strip()
                news_data.append((title, date, summary))

            # 딜레이 추가 (너무 빠른 요청 방지)
            time.sleep(1)

        return news_data

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []



In [ ]:
# 한달마다 데이터 뽑아오기

month_list = ['2023.01.01', '2023.02.01', '2023.03.01', '2023.04.01', '2023.05.01', '2023.06.01']
month_end_list = ['2023.01.31', '2023.02.28', '2023.03.31', '2023.04.30', '2023.05.31', '2023.06.30']


# company_name = "LG"
# start_date = "2023.10.01"
# end_date = "2023.10.31"


for month in range(6):
    news_df = pd.DataFrame(columns=['company', 'title', 'date', 'summary'])
    for company in tqdm(kospi_name):
        print(f"News for {company} from {month_list[month]} to {month_end_list[month]}:")
        # 만약 company의 30개정도 돌았으면, 10초정도 쉬어준다
        if len(news_df) % 300 == 0:
            time.sleep(10)
        news_data = get_naver_news_titles(company, month_list[month], month_end_list[month])
        for i, (title, date, summary) in enumerate(news_data, 1):
            print(f"{i}. {title} ({date})")
            news_df.loc[len(news_df)] = [company, title, date, summary]
    news_df.to_csv(f'뉴스크롤링{month}.csv', index=False, encoding='utf-8-sig')


news_df.to_csv('뉴스크롤링.csv', index=False, encoding='utf-8-sig')